In [1]:
import numpy as np

### Instantiate the prediction task

In [2]:
from jenga.tasks.income import IncomeEstimationTask

task = IncomeEstimationTask(seed=42)

### Task details

In this task, we try to predict the income level of a person (more or less than 50K dollars per years) from demographic and work-related data. It is often used as a proxy to study automated decision making for loan applications.

The original data is available in the UCI machine learning repository at https://archive.ics.uci.edu/ml/datasets/adult

In [3]:
task.train_data

,workclass,occupation,marital_status,education,hours_per_week,age
22582,Private,Sales,Never-married,11th,16,17
15224,Private,Adm-clerical,Never-married,HS-grad,48,34
12360,Private,Adm-clerical,Widowed,Some-college,40,64
2959,Self-emp-inc,Farming-fishing,Married-civ-spouse,10th,80,48
5076,Federal-gov,Other-service,Never-married,HS-grad,32,32
...,...,...,...,...,...,...
12682,Private,Adm-clerical,Never-married,HS-grad,35,19
10553,Private,Other-service,Married-civ-spouse,Bachelors,40,40
15135,Private,Machine-op-inspct,Married-civ-spouse,HS-grad,40,47
25392,Local-gov,Other-service,Divorced,Masters,40,57


### Train the provided baseline model

Jenga allows us to easily train and evaluate a logistic regression model for this task. Have a look at https://github.com/schelterlabs/jenga/blob/master/jenga/tasks/income.py if you want to know the details.


In [4]:
model = task.fit_baseline_model(task.train_data,task.train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    3.9s finished


In [5]:
y_pred = model.predict_proba(task.test_data)

f"The ROC AUC score on the test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the test data is 0.8799448422970204'

### Let's have a look at the test data

In [6]:
task.test_data

,workclass,occupation,marital_status,education,hours_per_week,age
8917,Private,Craft-repair,Married-civ-spouse,HS-grad,40,47
20837,Local-gov,Exec-managerial,Married-civ-spouse,Bachelors,40,58
17361,Private,Tech-support,Never-married,Bachelors,40,29
2954,Private,Machine-op-inspct,Never-married,HS-grad,40,26
21803,NaN,NaN,Never-married,Some-college,40,22
...,...,...,...,...,...,...
22218,Private,Adm-clerical,Divorced,HS-grad,43,46
23194,State-gov,Prof-specialty,Married-civ-spouse,Doctorate,45,37
27052,Self-emp-not-inc,Farming-fishing,Married-civ-spouse,HS-grad,56,47
11722,State-gov,Other-service,Married-civ-spouse,HS-grad,40,46


### Corruptions

Jenga provides a set of predefined data corruptions that we can use to simulate errors in the test data. We will simulate 'implicit missing values' in a column and see how this impacts the prediction quality.

Jenga supports much more predefined corruptions, have a look at https://github.com/schelterlabs/jenga/tree/master/jenga/corruptions.


In [7]:
from jenga.corruptions.generic import MissingValues

marital_status_corruption = MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN')


In [8]:
corrupted_test_data = marital_status_corruption.transform(task.test_data)
corrupted_test_data

,workclass,occupation,marital_status,education,hours_per_week,age
8917,Private,Craft-repair,BROKEN,HS-grad,40,47
20837,Local-gov,Exec-managerial,BROKEN,Bachelors,40,58
17361,Private,Tech-support,BROKEN,Bachelors,40,29
2954,Private,Machine-op-inspct,BROKEN,HS-grad,40,26
21803,NaN,NaN,BROKEN,Some-college,40,22
...,...,...,...,...,...,...
22218,Private,Adm-clerical,BROKEN,HS-grad,43,46
23194,State-gov,Prof-specialty,BROKEN,Doctorate,45,37
27052,Self-emp-not-inc,Farming-fishing,BROKEN,HS-grad,56,47
11722,State-gov,Other-service,BROKEN,HS-grad,40,46


In [9]:
y_pred = model.predict_proba(corrupted_test_data)

f"The ROC AUC score on the corrupted test data is {task.score_on_test_data(y_pred)}"

'The ROC AUC score on the corrupted test data is 0.8045116792453368'

### Jenga's evaluators

Jenga provides a set of evaluators which allow us to automate the evaluation of the impact of given data corruptions.

In [10]:
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator

evaluator = CorruptionImpactEvaluator(task)

In [11]:
corruptions = [
    MissingValues(column='marital_status', fraction=0.99, na_value='BROKEN'),
    MissingValues(column='age', fraction=0.05, na_value=-999),
]

### Run the evaluation of the corruptions with 10 repetitions 

In [12]:
num_repetitions = 10
results = evaluator.evaluate(model, num_repetitions, *corruptions)

0/20 (0.019395000000000273)
10/20 (0.19835700000000056)


### Investigate the impact on the predictive performance of the model

In [13]:
for validation_result in results:
    
    print(validation_result.corruption)
    print(f"""
     Score (AUC) on 
      clean data:     {validation_result.baseline_score}
      corrupted data: {np.mean(validation_result.corrupted_scores)}
     """)    
    print("\n")

MissingValues: {'column': 'marital_status', 'fraction': 0.99, 'sampling': 'MCAR', 'na_value': 'BROKEN'}

     Score (AUC) on 
      clean data:     0.8799448422970204
      corrupted data: 0.8038035552511928
     


MissingValues: {'column': 'age', 'fraction': 0.05, 'sampling': 'MCAR', 'na_value': -999}

     Score (AUC) on 
      clean data:     0.8799448422970204
      corrupted data: 0.843399127066248
     


